# Import library
run two cells below first

In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
import time
from datetime import datetime
import sys
import math

In [2]:
if 'holidays' not in sys.modules:
    !pip install holidays
import holidays

You should consider upgrading via the 'pip install --upgrade pip' command.


# Load three csv files
Summary: 937 unique drivers and 193502 unique rides in total

In [2]:
driver_df = pd.read_csv('driver_ids.csv')
ride_df = pd.read_csv('ride_ids.csv')
ride_timestamps_df = pd.read_csv('ride_timestamps.csv')

In [3]:
'''
Get the shape of each dataframe
'''
print('driver ids:',driver_df.shape)
display(driver_df.head())
print('ride ids:',ride_df.shape)
display(ride_df.head())
print('ride timestamps:',ride_timestamps_df.shape)
display(ride_timestamps_df.head())

driver ids: (937, 2)


,driver_id,driver_onboard_date
0,002be0ffdc997bd5c50703158b7c2491,2016-03-29 00:00:00
1,007f0389f9c7b03ef97098422f902e62,2016-03-29 00:00:00
2,011e5c5dfc5c2c92501b8b24d47509bc,2016-04-05 00:00:00
3,0152a2f305e71d26cc964f8d4411add9,2016-04-23 00:00:00
4,01674381af7edd264113d4e6ed55ecda,2016-04-29 00:00:00


ride ids: (193502, 5)


,driver_id,ride_id,ride_distance,ride_duration,ride_prime_time
0,002be0ffdc997bd5c50703158b7c2491,006d61cf7446e682f7bc50b0f8a5bea5,1811,327,50
1,002be0ffdc997bd5c50703158b7c2491,01b522c5c3a756fbdb12e95e87507eda,3362,809,0
2,002be0ffdc997bd5c50703158b7c2491,029227c4c2971ce69ff2274dc798ef43,3282,572,0
3,002be0ffdc997bd5c50703158b7c2491,034e861343a63ac3c18a9ceb1ce0ac69,65283,3338,25
4,002be0ffdc997bd5c50703158b7c2491,034f2e614a2f9fc7f1c2f77647d1b981,4115,823,100


ride timestamps: (970405, 3)


,ride_id,event,timestamp
0,00003037a262d9ee40e61b5c0718f7f0,requested_at,2016-06-13 09:39:19
1,00003037a262d9ee40e61b5c0718f7f0,accepted_at,2016-06-13 09:39:51
2,00003037a262d9ee40e61b5c0718f7f0,arrived_at,2016-06-13 09:44:31
3,00003037a262d9ee40e61b5c0718f7f0,picked_up_at,2016-06-13 09:44:33
4,00003037a262d9ee40e61b5c0718f7f0,dropped_off_at,2016-06-13 10:03:05


# Inspect Nan and abnormal values

In [4]:
'''
Nan value inspection
'''
print('driver ids info:------------------------------')
driver_df.info()
print('ride ids info:--------------------------------')
ride_df.info()
print('ride timestamps info:-------------------------')
ride_timestamps_df.info()
'''
ride_timestamps has one Nan value in the column timestamp
    TODO: delete this ride or fill it with an artificial value?
'''
display(ride_timestamps_df[ride_timestamps_df.isnull().any(axis=1)])

driver ids info:------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 937 entries, 0 to 936
Data columns (total 2 columns):
driver_id              937 non-null object
driver_onboard_date    937 non-null object
dtypes: object(2)
memory usage: 14.7+ KB
ride ids info:--------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193502 entries, 0 to 193501
Data columns (total 5 columns):
driver_id          193502 non-null object
ride_id            193502 non-null object
ride_distance      193502 non-null int64
ride_duration      193502 non-null int64
ride_prime_time    193502 non-null int64
dtypes: int64(3), object(2)
memory usage: 7.4+ MB
ride timestamps info:-------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 970405 entries, 0 to 970404
Data columns (total 3 columns):
ride_id      970405 non-null object
event        970405 non-null object
timestamp    970404 non-null object
dtypes: object(3)
memory usage: 22.2+ MB


,ride_id,event,timestamp
434222,72f0fa0bd86800e9da5c4dced32c8735,arrived_at,NaN


In [5]:
'''
Abnormal value inspection
'''
display(driver_df.describe())
display(ride_df.describe())
display(ride_timestamps_df.describe())

,driver_id,driver_onboard_date
count,937,937
unique,937,49
top,9494350df132e6748afca3bc5d138dcc,2016-04-05 00:00:00
freq,1,36


,ride_distance,ride_duration,ride_prime_time
count,193502.000000,193502.000000,193502.000000
mean,6955.218266,858.966099,17.305893
std,8929.444606,571.375818,30.825800
min,-2.000000,2.000000,0.000000
25%,2459.000000,491.000000,0.000000
50%,4015.000000,727.000000,0.000000
75%,7193.000000,1069.000000,25.000000
max,724679.000000,28204.000000,500.000000


,ride_id,event,timestamp
count,970405,970405,970404
unique,194081,5,865826
top,43b8fb03256e30a1a93db73d09a05b26,requested_at,2016-04-30 22:09:15
freq,5,194081,7


In [6]:
'''
TODO: Need to think about how to deal with this case, why will ride_distance <= 0?
TODO: the number of ride_id in ride_df and that of ride_timestamps doesn't fit (193502 vs 194081)
'''
abnormal_ride_df = ride_df[ride_df.ride_distance <= 0]
print(abnormal_ride_df.shape)
display(abnormal_ride_df.head())

(187, 5)


,driver_id,ride_id,ride_distance,ride_duration,ride_prime_time
3557,04d686ef2fba281e395992a28f14fdd3,c819e844b0c043786f1291e466478ca0,0,127,0
4221,0656192a402808805282e60761bda088,2f4fe99f74f1d71fdf2e8e18da9670c0,0,404,25
5120,07dd442e3e0b9f0f9b0d69c7b47cbb06,ad2b44316299477f2d6537922af22bf4,0,7,0
7144,0b631e16fa61f7321da18cf35a076d5f,1410283d27fdcf6c25a11f856ce5cde0,0,66,0
7713,0c02bd2b09f7193103279ab9b760b777,9fd795fdb616bd47c41e1ad07abbd6f3,0,23,50


In [7]:
'''
find overlap of driver_id between dirver_df and ride_df
TODO: some drivers don't have ride information--->delete? (937 vs 854)
'''
print(len(set(driver_df.driver_id.unique()).intersection(set(ride_df.driver_id.unique()))))

854


In [8]:
'''
find overlap of ride_id between ride_df and ride_timestamps_df
TODO: some rides don't have ride timestamps--->delete? (193502 vs 184819)
'''
print(len(set(ride_df.ride_id.unique()).intersection(set(ride_timestamps_df.ride_id.unique()))))

184819


# Merge all dfs to one df

In [9]:
'''
merge driver_df and ride_df (Get intersection based on driver_id)
'''
big_df = ride_df.merge(driver_df,left_on='driver_id',right_on='driver_id')
print(big_df.shape)
display(big_df.head())

(185891, 6)


,driver_id,ride_id,ride_distance,ride_duration,ride_prime_time,driver_onboard_date
0,002be0ffdc997bd5c50703158b7c2491,006d61cf7446e682f7bc50b0f8a5bea5,1811,327,50,2016-03-29 00:00:00
1,002be0ffdc997bd5c50703158b7c2491,01b522c5c3a756fbdb12e95e87507eda,3362,809,0,2016-03-29 00:00:00
2,002be0ffdc997bd5c50703158b7c2491,029227c4c2971ce69ff2274dc798ef43,3282,572,0,2016-03-29 00:00:00
3,002be0ffdc997bd5c50703158b7c2491,034e861343a63ac3c18a9ceb1ce0ac69,65283,3338,25,2016-03-29 00:00:00
4,002be0ffdc997bd5c50703158b7c2491,034f2e614a2f9fc7f1c2f77647d1b981,4115,823,100,2016-03-29 00:00:00


In [10]:
# get overlapped ride_id between big_df and ride_timestamps_df
big_df = big_df[big_df['ride_id'].isin(ride_timestamps_df.ride_id.unique())]
big_df.reset_index(drop=True,inplace=True)
print(big_df.shape)
display(big_df.head())

(184209, 6)


,driver_id,ride_id,ride_distance,ride_duration,ride_prime_time,driver_onboard_date
0,002be0ffdc997bd5c50703158b7c2491,006d61cf7446e682f7bc50b0f8a5bea5,1811,327,50,2016-03-29 00:00:00
1,002be0ffdc997bd5c50703158b7c2491,01b522c5c3a756fbdb12e95e87507eda,3362,809,0,2016-03-29 00:00:00
2,002be0ffdc997bd5c50703158b7c2491,029227c4c2971ce69ff2274dc798ef43,3282,572,0,2016-03-29 00:00:00
3,002be0ffdc997bd5c50703158b7c2491,034e861343a63ac3c18a9ceb1ce0ac69,65283,3338,25,2016-03-29 00:00:00
4,002be0ffdc997bd5c50703158b7c2491,034f2e614a2f9fc7f1c2f77647d1b981,4115,823,100,2016-03-29 00:00:00


In [11]:
start = time.time()
# for each unique ride id in big_df
for idx in range(big_df.shape[0]):
    rideid = big_df.iloc[idx]['ride_id']
    # first find rideid timestamps info in ride_timestamps_df
    target = ride_timestamps_df[ride_timestamps_df.ride_id == rideid]
    # for each (event,timestamp) pair
    for (e,t) in zip(list(target.event),list(target.timestamp)):
        big_df.at[idx,e] = t
    # double check index
    if big_df[big_df.ride_id == rideid]['requested_at'].values[0] != \
    ride_timestamps_df[ride_timestamps_df.ride_id == rideid].iloc[0,-1]:
        print(idx)
print('duration:',(time.time()-start)/3600,'hrs')

duration: 7.626359305050638 hrs


In [12]:
big_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184209 entries, 0 to 184208
Data columns (total 11 columns):
driver_id              184209 non-null object
ride_id                184209 non-null object
ride_distance          184209 non-null int64
ride_duration          184209 non-null int64
ride_prime_time        184209 non-null int64
driver_onboard_date    184209 non-null object
requested_at           184209 non-null object
accepted_at            184209 non-null object
arrived_at             184208 non-null object
picked_up_at           184209 non-null object
dropped_off_at         184209 non-null object
dtypes: int64(3), object(8)
memory usage: 15.5+ MB


In [13]:
# saved for future use
big_df.to_csv('merged_big_driver_ride_df.csv',index=False)

# Start to work on calculating extra variables
If already have file 'merged_big_driver_ride_df.csv', directly start running code below

In [18]:
merged_big_df = pd.read_csv('merged_big_driver_ride_df.csv')
print(merged_big_df.shape)
display(merged_big_df.head())

(184209, 11)


,driver_id,ride_id,ride_distance,ride_duration,ride_prime_time,driver_onboard_date,requested_at,accepted_at,arrived_at,picked_up_at,dropped_off_at
0,002be0ffdc997bd5c50703158b7c2491,006d61cf7446e682f7bc50b0f8a5bea5,1811,327,50,2016-03-29 00:00:00,2016-04-23 02:13:50,2016-04-23 02:14:15,2016-04-23 02:16:36,2016-04-23 02:16:40,2016-04-23 02:22:07
1,002be0ffdc997bd5c50703158b7c2491,01b522c5c3a756fbdb12e95e87507eda,3362,809,0,2016-03-29 00:00:00,2016-03-29 19:00:49,2016-03-29 19:00:52,2016-03-29 19:03:57,2016-03-29 19:04:01,2016-03-29 19:17:30
2,002be0ffdc997bd5c50703158b7c2491,029227c4c2971ce69ff2274dc798ef43,3282,572,0,2016-03-29 00:00:00,2016-06-21 11:56:31,2016-06-21 11:56:39,2016-06-21 12:01:32,2016-06-21 12:01:35,2016-06-21 12:11:07
3,002be0ffdc997bd5c50703158b7c2491,034e861343a63ac3c18a9ceb1ce0ac69,65283,3338,25,2016-03-29 00:00:00,2016-05-19 09:15:29,2016-05-19 09:15:33,2016-05-19 09:18:20,2016-05-19 09:18:20,2016-05-19 10:13:58
4,002be0ffdc997bd5c50703158b7c2491,034f2e614a2f9fc7f1c2f77647d1b981,4115,823,100,2016-03-29 00:00:00,2016-04-20 22:05:30,2016-04-20 22:05:32,2016-04-20 22:07:03,2016-04-20 22:07:02,2016-04-20 22:20:45


In [19]:
# '''
# validate the correctness of combined df by randomly selecting ride ids to verify (random checking)
# '''
# ids = test1.ride_id
# i = np.random.choice(ids,10)
# for x in i:
#     display(test1[test1.ride_id == x])
#     display(ride_timestamps_df[ride_timestamps_df.ride_id == x])

## Get new variables related to ride info
time of day: 0-6(midnight); 6-9(morning rush); 9-16(normal day); 16-19(evening rush); 19-24(fun) <br>
season: {0: Spring, 1: Summer, 2: Autumn, 3: Winter}

In [20]:
# variables related to ride
def add_vars(row):
    # convert time to datetime
    # source: https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior
    for i in range(5,len(row)):
        if type(row[i]) != float:
            row[i] = datetime.strptime(row[i], '%Y-%m-%d %H:%M:%S')
    # get speed
    row['speed(m/s)'] = row['ride_distance'] / row['ride_duration']
    # holiday? reference: https://stackoverflow.com/questions/2394235/detecting-a-us-holiday
    row['holiday'] = int(row['requested_at'] in holidays.US())
    # requested_at time is weekend? (0-6 --> Mon-Sun)
    row['weekend'] = int(row['requested_at'].weekday() > 4)
    # time of day (6-12/12-15/16-20/20-24/24-6)
    # {0: [6,9), 1: [9,16), 2: [16,19), 3: [19,24), 4: [0,6)}
    if 6 <= row['requested_at'].hour < 9:
        row['time of day'] = 0
    elif 9 <= row['requested_at'].hour < 16:
        row['time of day'] = 1
    elif 16 <= row['requested_at'].hour < 19:
        row['time of day'] = 2
    elif 19 <= row['requested_at'].hour < 24:
        row['time of day'] = 3
    else:
        row['time of day'] = 4
    # season (12-2 Winter/3-5 Spring/6-8 Summer/9-11 Autumn)
    # {0: Spring, 1: Summer, 2: Autumn, 3: Winter}
    if 3 <= row['requested_at'].month <= 5:
        row['season'] = 0
    elif 6 <= row['requested_at'].month <= 8:
        row['season'] = 1
    elif 9 <= row['requested_at'].month <= 11:
        row['season'] = 2
    else:
        row['season'] = 3
    # time spent from requested_at to arrived_at (efficiency of picking up a customer)
    if type(row['arrived_at']) != float:
        row['time spent to arrive at the customer(minutes)'] = (row['arrived_at']-\
                                                                row['requested_at']).total_seconds()/60
    else:
        row['time spent to arrive at the customer(minutes)'] = (row['picked_up_at']-\
                                                                row['requested_at']).total_seconds()/60
    return row

start = time.time()
rides_added_vars_df = merged_big_df.apply(add_vars,axis=1)
print('duration:',(time.time() - start)/60,'minutes')

duration: 15.989163434505462 minutes


In [21]:
print(rides_added_vars_df.shape)
display(rides_added_vars_df.head())

(184209, 17)


,driver_id,ride_id,ride_distance,ride_duration,ride_prime_time,driver_onboard_date,requested_at,accepted_at,arrived_at,picked_up_at,dropped_off_at,speed(m/s),holiday,weekend,time of day,season,time spent to arrive at the customer(minutes)
0,002be0ffdc997bd5c50703158b7c2491,006d61cf7446e682f7bc50b0f8a5bea5,1811,327,50,2016-03-29,2016-04-23 02:13:50,2016-04-23 02:14:15,2016-04-23 02:16:36,2016-04-23 02:16:40,2016-04-23 02:22:07,5.538226,0,1,4,0,2.766667
1,002be0ffdc997bd5c50703158b7c2491,01b522c5c3a756fbdb12e95e87507eda,3362,809,0,2016-03-29,2016-03-29 19:00:49,2016-03-29 19:00:52,2016-03-29 19:03:57,2016-03-29 19:04:01,2016-03-29 19:17:30,4.155748,0,0,3,0,3.133333
2,002be0ffdc997bd5c50703158b7c2491,029227c4c2971ce69ff2274dc798ef43,3282,572,0,2016-03-29,2016-06-21 11:56:31,2016-06-21 11:56:39,2016-06-21 12:01:32,2016-06-21 12:01:35,2016-06-21 12:11:07,5.737762,0,0,1,1,5.016667
3,002be0ffdc997bd5c50703158b7c2491,034e861343a63ac3c18a9ceb1ce0ac69,65283,3338,25,2016-03-29,2016-05-19 09:15:29,2016-05-19 09:15:33,2016-05-19 09:18:20,2016-05-19 09:18:20,2016-05-19 10:13:58,19.557519,0,0,1,0,2.850000
4,002be0ffdc997bd5c50703158b7c2491,034f2e614a2f9fc7f1c2f77647d1b981,4115,823,100,2016-03-29,2016-04-20 22:05:30,2016-04-20 22:05:32,2016-04-20 22:07:03,2016-04-20 22:07:02,2016-04-20 22:20:45,5.000000,0,0,3,0,1.550000


In [9]:
# saved for future use
rides_added_vars_df.to_csv('added_variables_rides_info.csv',index=False)

## Get new variables related to drivers

In [22]:
drivers_df = pd.DataFrame(rides_added_vars_df.driver_id.unique())
drivers_df.rename(columns={0:'driver_id'},inplace=True)
drivers_df.reset_index(drop=True,inplace=True)
print(drivers_df.shape)
display(drivers_df.head())

(837, 1)


,driver_id
0,002be0ffdc997bd5c50703158b7c2491
1,007f0389f9c7b03ef97098422f902e62
2,011e5c5dfc5c2c92501b8b24d47509bc
3,0152a2f305e71d26cc964f8d4411add9
4,01674381af7edd264113d4e6ed55ecda


In [24]:
def get_fare(driver_rides):
    total_fare = 0
    for (distance,duration,prime) in zip(driver_rides['ride_distance'].values,\
                                         driver_rides['ride_duration'].values,\
                                         driver_rides['ride_prime_time'].values):
        total_fare += (1 + prime/100)*(min(max(5,(2 + 1.15*distance*0.00062 + 0.22*duration/60 + 1.75)),400))
    return total_fare

def add_driver_vars(row):
    # first find all rides under driverid
    rides = rides_added_vars_df[rides_added_vars_df.driver_id == row['driver_id']]
    # Percentage of rides completed in prime time among all rides for each driver
    row['prime time rides percentage'] = rides[rides.ride_prime_time != 0].shape[0] / rides.shape[0]
    # Average active time per day for each driver (total duration / total days)
    sorted_days = rides.groupby('requested_at', as_index=False).mean()['requested_at']
    days = len(sorted_days.dt.normalize().unique())
    row['average daily active time(hrs/day)'] = rides.ride_duration.sum() / (3600*days)
    # Average fare(daily/per ride/monthly) received for each driver
    total_fare = get_fare(rides)
    row['gross fare(over all rides)'] = total_fare
    row['average daily fare'] = total_fare / days
    row['average fare per ride'] = total_fare / rides.shape[0]
    number_of_months = len(set([(x.year,x.month) for x in sorted_days]))
    row['average monthly fare'] = total_fare / number_of_months
    # Total number of rides
    row['total rides'] = rides.shape[0]
    # Number of abnormal rides (ride_distance <= 0)
    row['number of abnormal rides'] = rides[rides.ride_distance <= 0].shape[0]
    # Ride completion rate (1 - (# of abnormal / total rides))
    row['completion rate'] = 1 - (row['number of abnormal rides'] / row['total rides'])
    # Unique days of work
    row['active days'] = days
    # Average time spent on each ride (requested_at --> arrived_at) in minutes
    row['average arriving time(minutes)'] = rides['time spent to arrive at the customer(minutes)'].mean()
    return row

start = time.time()
drivers_added_vars_df = drivers_df.apply(add_driver_vars,axis=1)
print('duration:',(time.time() - start)/60,'minutes')

duration: 0.41034599939982097 minutes


In [25]:
print(drivers_added_vars_df.shape)
display(drivers_added_vars_df.head())

(837, 12)


,driver_id,prime time rides percentage,average daily active time(hrs/day),gross fare(over all rides),average daily fare,average fare per ride,average monthly fare,total rides,number of abnormal rides,completion rate,active days,average arriving time(minutes)
0,002be0ffdc997bd5c50703158b7c2491,0.397112,1.097411,3651.957780,65.213532,13.183963,912.989445,277,0,1.0,56,3.023285
1,007f0389f9c7b03ef97098422f902e62,0.387097,0.474468,332.355469,27.696289,10.721144,83.088867,31,0,1.0,12,3.150000
2,011e5c5dfc5c2c92501b8b24d47509bc,0.470588,0.676042,493.765123,41.147094,14.522504,164.588374,34,0,1.0,12,3.842157
3,0152a2f305e71d26cc964f8d4411add9,0.251309,1.211951,2644.233047,66.105826,13.844152,881.411016,191,0,1.0,40,4.145462
4,01674381af7edd264113d4e6ed55ecda,0.264000,2.482243,5457.997548,136.449939,14.554660,1819.332516,375,0,1.0,40,3.816933


In [26]:
# saved for future use
drivers_added_vars_df.to_csv('added_variables_drivers_info.csv',index=False)

# Start to construct models

In [27]:
'''
rides info combined dataframe
'''
big_rides_info = pd.read_csv('added_variables_rides_info.csv')
print(big_rides_info.shape)
display(big_rides_info.head())

'''
drivers info combined dataframe
'''
big_drivers_info = pd.read_csv('added_variables_drivers_info.csv')
print(big_drivers_info.shape)
display(big_drivers_info.head())

(184209, 17)


,driver_id,ride_id,ride_distance,ride_duration,ride_prime_time,driver_onboard_date,requested_at,accepted_at,arrived_at,picked_up_at,dropped_off_at,speed(m/s),holiday,weekend,time of day,season,time spent to arrive at the customer(minutes)
0,002be0ffdc997bd5c50703158b7c2491,006d61cf7446e682f7bc50b0f8a5bea5,1811,327,50,2016-03-29 00:00:00,2016-04-23 02:13:50,2016-04-23 02:14:15,2016-04-23 02:16:36,2016-04-23 02:16:40,2016-04-23 02:22:07,5.538226,0,1,4,0,2.766667
1,002be0ffdc997bd5c50703158b7c2491,01b522c5c3a756fbdb12e95e87507eda,3362,809,0,2016-03-29 00:00:00,2016-03-29 19:00:49,2016-03-29 19:00:52,2016-03-29 19:03:57,2016-03-29 19:04:01,2016-03-29 19:17:30,4.155748,0,0,3,0,3.133333
2,002be0ffdc997bd5c50703158b7c2491,029227c4c2971ce69ff2274dc798ef43,3282,572,0,2016-03-29 00:00:00,2016-06-21 11:56:31,2016-06-21 11:56:39,2016-06-21 12:01:32,2016-06-21 12:01:35,2016-06-21 12:11:07,5.737762,0,0,1,1,5.016667
3,002be0ffdc997bd5c50703158b7c2491,034e861343a63ac3c18a9ceb1ce0ac69,65283,3338,25,2016-03-29 00:00:00,2016-05-19 09:15:29,2016-05-19 09:15:33,2016-05-19 09:18:20,2016-05-19 09:18:20,2016-05-19 10:13:58,19.557519,0,0,1,0,2.850000
4,002be0ffdc997bd5c50703158b7c2491,034f2e614a2f9fc7f1c2f77647d1b981,4115,823,100,2016-03-29 00:00:00,2016-04-20 22:05:30,2016-04-20 22:05:32,2016-04-20 22:07:03,2016-04-20 22:07:02,2016-04-20 22:20:45,5.000000,0,0,3,0,1.550000


(837, 12)


,driver_id,prime time rides percentage,average daily active time(hrs/day),gross fare(over all rides),average daily fare,average fare per ride,average monthly fare,total rides,number of abnormal rides,completion rate,active days,average arriving time(minutes)
0,002be0ffdc997bd5c50703158b7c2491,0.397112,1.097411,3651.957780,65.213532,13.183963,912.989445,277,0,1.0,56,3.023285
1,007f0389f9c7b03ef97098422f902e62,0.387097,0.474468,332.355469,27.696289,10.721144,83.088867,31,0,1.0,12,3.150000
2,011e5c5dfc5c2c92501b8b24d47509bc,0.470588,0.676042,493.765123,41.147094,14.522504,164.588374,34,0,1.0,12,3.842157
3,0152a2f305e71d26cc964f8d4411add9,0.251309,1.211951,2644.233047,66.105826,13.844152,881.411016,191,0,1.0,40,4.145462
4,01674381af7edd264113d4e6ed55ecda,0.264000,2.482243,5457.997548,136.449939,14.554660,1819.332516,375,0,1.0,40,3.816933
